<a href="https://colab.research.google.com/github/OmarMohy/LangChain-Playground/blob/main/LAB_2_Calculator_App.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Calculator Agent

This Calculator agent has the following tools:

- **Tool 1: Parsing the equation:** It takes a math problem and it extracts the corresponding equation (e.g., '5+2') using an LLM.
- **Tool 2: Performing the operation:** It takes an equation and uses the calculator API to solve it (e.g., '5+2=7').
- **Tool 3: Generating the answer:** It takes the math problem and the solved equation, and it uses an LLM to formulate an solution to the problem.


## Installations

To get started, we first need to set up our environment by installing the necessary libraries. This step ensures that we have all the dependencies required to build our calculator agent. We'll be using LangChain as our primary framework, which provides the tools and components needed to create and connect our agent's different functionalities. We'll also install the langchain-google-genai library to leverage a powerful Google model, like gemini-1.5-flash, as the agent's core engine.

In [2]:
!pip install -q langchain langchain-google-genai google-generativeai langgraph wikipedia langchain_community

print("Libraries installed: langchain, langchain-google-genai, google-generativeai, langgraph, wikipedia, langchain_community")
print("Libraries installed: langchain, langchain-google-genai, google-generativeai, langgraph, wikipedia")

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.3/53.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 713.3/713.3 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 234.9/234.9 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 2.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires google-auth==2.43.0, but you have google-auth 2.47.0 which is incompatible.
google-colab 1.0.0 requires requests==2.32.4, but 

In [3]:
import os
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, ToolMessage
import time # For slight delays to help with rate limits

# Retrieve the API key from Colab secrets
try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY
    print("Google API Key loaded from Colab secrets!")
except userdata.SecretNotFoundError:
    print("Error: GOOGLE_API_KEY not found in Colab secrets.")
    print("Please ensure you have set a secret named 'GOOGLE_API_KEY' in Colab.")
    print("You can get a free key from https://aistudio.google.com/.")
    raise ValueError("API Key not configured. Please set GOOGLE_API_KEY in Colab secrets.")
except Exception as e:
    print(f"An unexpected error occurred while loading API Key: {e}")
    raise

# Initialize a base LLM that all our agents will use as their "brain"
# We'll stick to gemini-1.5-flash for its speed and free tier.
base_llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.1, convert_system_message_to_human=True)

print(f"Base LLM initialized: {base_llm.model}")
print("This base LLM will power our specialized agents for understanding and generating text.")

ModuleNotFoundError: No module named 'langchain.prompts'

In [5]:
import os
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, ToolMessage
import time # For slight delays to help with rate limits

# Retrieve the API key from Colab secrets
try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY
    print("Google API Key loaded from Colab secrets!")
except userdata.SecretNotFoundError:
    print("Error: GOOGLE_API_KEY not found in Colab secrets.")
    print("Please ensure you have set a secret named 'GOOGLE_API_KEY' in Colab.")
    print("You can get a free key from https://aistudio.google.com/.")
    raise ValueError("API Key not configured. Please set GOOGLE_API_KEY in Colab secrets.")
except Exception as e:
    print(f"An unexpected error occurred while loading API Key: {e}")
    raise

# Initialize a base LLM that all our agents will use as their "brain"
# We'll stick to gemini-1.5-flash for its speed and free tier.
base_llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.1, convert_system_message_to_human=True)

print(f"Base LLM initialized: {base_llm.model}")
print("This base LLM will power our specialized agents for understanding and generating text.")

Google API Key loaded from Colab secrets!
Base LLM initialized: gemini-1.5-flash
This base LLM will power our specialized agents for understanding and generating text.


In [ ]:
import warnings

# Filter out the specific UserWarning from langchain_google_genai
warnings.filterwarnings("ignore", message="Convert_system_message_to_human will be deprecated!", module="langchain_google_genai")

print("Warning for 'Convert_system_message_to_human will be deprecated!' is now suppressed.")

Warning for 'Convert_system_message_to_human will be deprecated!' is now suppressed.


## Tool 1: Parsing the equation from the math problem

The first challenge in building a calculator agent is to translate a user's natural language question into a formal mathematical expression that a computer can understand. In this section, we use a LangChain prompt template to guide our Large Language Model (LLM) to extract the equation. The prompt is carefully designed to ask the LLM to identify the core mathematical operation(s) and numbers from the user's query, transforming a sentence like "What is 10 plus 5?" into a simple equation like "10 + 5".

In [ ]:
def get_equation_from_word_problem(word_problem: str) -> str:
    """
    Uses an LLM to extract the mathematical equation from a word problem.

    Args:
        word_problem: The math word problem as a string.

    Returns:
        A string representing the extracted mathematical equation, ready for a calculator.
    """
    # Define the prompt for the LLM
    prompt = ChatPromptTemplate.from_messages([
        ("system", "Extract the mathematical equation from the word problem. Provide ONLY the equation itself, in a format suitable for direct input into a calculator or mathematical solver. For example, if the problem is 'What is 5 plus 3?', output '5 + 3'."),
        ("user", "Extract the equation from the following word problem: {word_problem}")
    ])

    # Chain the prompt and the LLM
    chain = prompt | base_llm

    # Invoke the chain with the word problem
    response = chain.invoke({"word_problem": word_problem})

    # Return the extracted equation, ensuring no leading/trailing whitespace
    return response.content.strip()

# Example usage:
word_problem = "I have three apples and ate 1, how many do I have"
equation = get_equation_from_word_problem(word_problem)
print(f"Word Problem: {word_problem}")
print(f"Extracted Equation: {equation}")

word_problem_2 = "If a car travels at 60 miles per hour for 2 hours, how far does it travel?"
equation_2 = get_equation_from_word_problem(word_problem_2)
print(f"Word Problem: {word_problem_2}")
print(f"Extracted Equation: {equation_2}")

word_problem_3 = "What is the sum of 10 and 25 divided by 5?"
equation_3 = get_equation_from_word_problem(word_problem_3)
print(f"Word Problem: {word_problem_3}")
print(f"Extracted Equation: {equation_3}")

Word Problem: I have three apples and ate 1, how many do I have
Extracted Equation: 3 - 1
Word Problem: If a car travels at 60 miles per hour for 2 hours, how far does it travel?
Extracted Equation: 60 * 2
Word Problem: What is the sum of 10 and 25 divided by 5?
Extracted Equation: (10 + 25) / 5


## Tool 2: Performing the mathematical operation

Once we have extracted the equation, we need a way to perform the calculation. An LLM on its own isn't reliable for precise mathematical operations. This is where tools come in. In this section, we'll create a dedicated Python tool to perform the calculation accurately. This tool, which we can call calculate_equation, takes the extracted equation as input and uses Python's built-in functionality to compute the result. This separates the creative task of parsing the user's request from the analytical task of solving the math problem.

In [ ]:
import math # Import math for more complex operations if needed

def calculate_equation(equation: str) -> float:
    """
    Evaluates a mathematical equation string and returns the numerical result.

    Args:
        equation: The mathematical equation as a string (e.g., '4*2-3').

    Returns:
        The numerical result of the equation.
    """
    try:
        # WARNING: Using eval() can be dangerous if the input is not trusted.
        # In a real application, you would use a secure math expression parser or a dedicated calculator API.
        # Provide access to math functions within eval's scope
        result = eval(equation, {"__builtins__": None}, {"math": math, "sqrt": math.sqrt, "log": math.log})
        return float(result)
    except Exception as e:
        print(f"Error evaluating equation '{equation}': {e}")
        return float('nan') # Return NaN for invalid equations

# Example usage:
equation_to_calculate = "3 - 1"
result = calculate_equation(equation_to_calculate)
print(f"Equation: {equation_to_calculate}")
print(f"Result: {result}")

equation_to_calculate_2 = "60 * 2"
result_2 = calculate_equation(equation_to_calculate_2)
print(f"Equation: {equation_to_calculate_2}")
print(f"Result: {result_2}")

equation_to_calculate_3 = "(10 + 25) / 5"
result_3 = calculate_equation(equation_to_calculate_3)
print(f"Equation: {equation_to_calculate_3}")
print(f"Result: {result_3}")

equation_to_calculate_4 = "sqrt(16) + log(10)" # Example with math functions
result_4 = calculate_equation(equation_to_calculate_4)
print(f"Equation: {equation_to_calculate_4}")
print(f"Result: {result_4}")

Equation: 3 - 1
Result: 2.0
Equation: 60 * 2
Result: 120.0
Equation: (10 + 25) / 5
Result: 7.0
Equation: sqrt(16) + log(10)
Result: 6.302585092994046


## Tool 3: Generating an answer

The final step is to combine the calculated result with the original query to formulate a human-friendly answer. We use another LangChain prompt template for this. The prompt takes the user's original question and the numerical result from our calculator tool and instructs the LLM to generate a complete sentence. For example, it will turn a result of 25 and the original question "What is 5 times 5?" into the final answer, "5 times 5 is 25."

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

def generate_answer_sentence(word_problem: str, equation: str, result: float) -> str:
    """
    Uses an LLM to generate a natural language sentence answering the word problem,
    given the original problem, the extracted equation, and the calculated result.

    Args:
        word_problem: The original math word problem.
        equation: The extracted mathematical equation.
        result: The numerical result of the equation.

    Returns:
        A string containing a natural language answer to the word problem.
    """
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a helpful assistant that provides the answer to a math word problem in a clear and concise sentence. Given the original problem, the equation used, and the result, formulate a sentence that answers the original question."),
        ("user", "Word Problem: {word_problem}\nEquation: {equation}\nResult: {result}\n\nProvide the answer in a sentence.")
    ])

    chain = prompt | base_llm

    response = chain.invoke({"word_problem": word_problem, "equation": equation, "result": result})

    return response.content.strip()

# Example usage (assuming you have called the previous functions to get equation and result):
word_problem_example = "I have three apples and ate 1, how many do I have"
equation_example = "3 - 1" # This would come from get_equation_from_word_problem
result_example = 2.0 # This would come from calculate_equation

answer_sentence = generate_answer_sentence(word_problem_example, equation_example, result_example)
print(f"Word Problem: {word_problem_example}")
print(f"Answer: {answer_sentence}")

word_problem_example_2 = "If a car travels at 60 miles per hour for 2 hours, how far does it travel?"
equation_example_2 = "60 * 2"
result_example_2 = 120.0

answer_sentence_2 = generate_answer_sentence(word_problem_example_2, equation_example_2, result_example_2)
print(f"Word Problem: {word_problem_example_2}")
print(f"Answer: {answer_sentence_2}")

Word Problem: I have three apples and ate 1, how many do I have
Answer: You have 2 apples left.
Word Problem: If a car travels at 60 miles per hour for 2 hours, how far does it travel?
Answer: The car travels 120 miles.


In [ ]:
word_problem = "I have ten apples and ate one of them, how many do I have"
print(f"Word Problem: {word_problem}")
equation = get_equation_from_word_problem(word_problem)
print(f"Extracted Equation: {equation}")
result_from_equation = calculate_equation(equation)
print(f"Calculated Result from Equation: {result_from_equation}")
answer_sentence = generate_answer_sentence(word_problem, equation, result_from_equation)
print(f"Answer Sentence: {answer_sentence}")

Word Problem: I have ten apples and ate one of them, how many do I have
Extracted Equation: 10 - 1
Calculated Result from Equation: 9.0
Answer Sentence: You have 9 apples left.


## Whole agentic workflow

This section brings everything together. We'll use LangChain Expression Language (LCEL) to connect the different components we’ve built—the prompt to extract the equation, the LLM, and the calculator tool. We’ll chain these elements together into a single, seamless workflow. This chain represents our complete agent, which can now take a user query, parse it, perform a calculation, and generate a final response in one fluid motion.

In [ ]:
def solve_word_problem(word_problem: str) -> str:
    """
    Solves a math word problem by extracting the equation, calculating the result,
    and generating a natural language answer sentence.

    Args:
        word_problem: The math word problem as a string.

    Returns:
        A string containing the natural language answer to the word problem,
        or an error message if the problem could not be solved.
    """
    # 1. Extract the equation
    equation = get_equation_from_word_problem(word_problem)
    print(f"Extracted Equation: {equation}")

    if not equation:
        return "Could not extract a valid equation from the word problem."

    # 2. Calculate the result
    result = calculate_equation(equation)
    print(f"Calculated Result: {result}")

    if math.isnan(result):
        return f"Could not calculate the result for the equation: {equation}"

    # 3. Generate the answer sentence
    answer_sentence = generate_answer_sentence(word_problem, equation, result)

    return answer_sentence

# Example usage:
word_problem_to_solve = "I had 10 cookies and ate 4. My friend gave me 2 more. How many cookies do I have now?"
final_answer = solve_word_problem(word_problem_to_solve)
print(f"\nWord Problem: {word_problem_to_solve}")
print(f"Final Answer: {final_answer}")

word_problem_to_solve_2 = "The area of a rectangle is 25 square meters. If the width is 5 meters, what is the length?"
final_answer_2 = solve_word_problem(word_problem_to_solve_2)
print(f"\nWord Problem: {word_problem_to_solve_2}")
print(f"Final Answer: {final_answer_2}")

word_problem_to_solve_3 = "What is the square root of 81?"
final_answer_3 = solve_word_problem(word_problem_to_solve_3)
print(f"\nWord Problem: {word_problem_to_solve_3}")
print(f"Final Answer: {final_answer_3}")

Extracted Equation: 10 - 4 + 2
Calculated Result: 8.0

Word Problem: I had 10 cookies and ate 4. My friend gave me 2 more. How many cookies do I have now?
Final Answer: You now have 8 cookies.
Extracted Equation: 25/5
Calculated Result: 5.0

Word Problem: The area of a rectangle is 25 square meters. If the width is 5 meters, what is the length?
Final Answer: The length of the rectangle is 5 meters.
Extracted Equation: √81
Error evaluating equation '√81': invalid character '√' (U+221A) (<string>, line 1)
Calculated Result: nan

Word Problem: What is the square root of 81?
Final Answer: Could not calculate the result for the equation: √81


## Test Cases

To ensure our calculator agent works as expected, we'll test it with various examples. This section demonstrates how the complete agent workflow handles different types of mathematical questions, from simple arithmetic to word problems. Running these tests helps us verify that our agent can successfully parse queries, utilize its calculator tool, and provide accurate answers, proving that our agent is robust and reliable.

In [ ]:
# Test with a multiplication and addition problem
word_problem_test_1 = "If I buy 5 apples at 2 dollars each and 3 oranges at 1 dollar each, how much do I spend in total?"
final_answer_test_1 = solve_word_problem(word_problem_test_1)
print(f"\nWord Problem: {word_problem_test_1}")
print(f"Final Answer: {final_answer_test_1}")

# Test with a percentage problem
word_problem_test_2 = "What is 20% of 150?"
final_answer_test_2 = solve_word_problem(word_problem_test_2)
print(f"\nWord Problem: {word_problem_test_2}")
print(f"Final Answer: {final_answer_test_2}")

# Test with a problem that might require parentheses
word_problem_test_3 = "Calculate the result of adding 7 to 3 and then multiplying by 4."
final_answer_test_3 = solve_word_problem(word_problem_test_3)
print(f"\nWord Problem: {word_problem_test_3}")
print(f"Final Answer: {final_answer_test_3}")

# Test with a division problem
word_problem_test_4 = "If 100 items are divided equally among 4 groups, how many items are in each group?"
final_answer_test_4 = solve_word_problem(word_problem_test_4)
print(f"\nWord Problem: {word_problem_test_4}")
print(f"Final Answer: {final_answer_test_4}")

Extracted Equation: (5*2) + (3*1)
Calculated Result: 13.0

Word Problem: If I buy 5 apples at 2 dollars each and 3 oranges at 1 dollar each, how much do I spend in total?
Final Answer: You spend a total of $13.
Extracted Equation: 0.20*150
Calculated Result: 30.0

Word Problem: What is 20% of 150?
Final Answer: 20% of 150 is 30.
Extracted Equation: (7+3)*4
Calculated Result: 40.0

Word Problem: Calculate the result of adding 7 to 3 and then multiplying by 4.
Final Answer: Adding 7 to 3 and then multiplying by 4 results in 40.
Extracted Equation: 100/4
Calculated Result: 25.0

Word Problem: If 100 items are divided equally among 4 groups, how many items are in each group?
Final Answer: Each group will have 25 items.


# **Playaround**

In [ ]:
def get_equation_from_word_problem(word_problem: str) -> str:
    """
    Uses an LLM to extract the mathematical equation from a word problem.

    Args:
        word_problem: The math word problem as a string.

    Returns:
        A string representing the extracted mathematical equation, ready for a calculator.
    """
    # Define the prompt for the LLM
    prompt = ChatPromptTemplate.from_messages([
        ("system", "Extract the mathematical equation from the word problem. Provide ONLY the equation itself, in a format suitable for direct input into a calculator or mathematical solver. For example, if the problem is 'What is 5 plus 3?', output '5 + 3'."),
        ("user", "Extract the equation from the following word problem: {word_problem}")
    ])

    # Chain the prompt and the LLM
    chain = prompt | base_llm

    # Invoke the chain with the word problem
    response = chain.invoke({"word_problem": word_problem})

    # Return the extracted equation, ensuring no leading/trailing whitespace
    return response.content.strip()

# Example usage:
word_problem = "I have three apples and ate 1, how many do I have"
equation = get_equation_from_word_problem(word_problem)
print(f"Word Problem: {word_problem}")
print(f"Extracted Equation: {equation}")

In [ ]:
from langchain_